### Multivariate Linear Regression (다변량 선형 회귀)
- 하나의 종속변수와 여러 독립변수 사이의 관계를 분석하는 기법이다.

In [1]:
import pandas as pd

a_df = pd.read_csv('./datasets/advertising.csv')
a_df

Unnamed: 0     TV  Radio  Newspaper  Sales
0             1  230.1   37.8       69.2   22.1
1             2   44.5   39.3       45.1   10.4
2             3   17.2   45.9       69.3    9.3
3             4  151.5   41.3       58.5   18.5
4             5  180.8   10.8       58.4   12.9
..          ...    ...    ...        ...    ...
195         196   38.2    3.7       13.8    7.6
196         197   94.2    4.9        8.1    9.7
197         198  177.0    9.3        6.4   12.8
198         199  283.6   42.0       66.2   25.5
199         200  232.1    8.6        8.7   13.4

[200 rows x 5 columns]

In [2]:
pre_a_df = a_df.drop(labels=['Unnamed: 0'], axis=1)
pre_a_df

TV  Radio  Newspaper  Sales
0    230.1   37.8       69.2   22.1
1     44.5   39.3       45.1   10.4
2     17.2   45.9       69.3    9.3
3    151.5   41.3       58.5   18.5
4    180.8   10.8       58.4   12.9
..     ...    ...        ...    ...
195   38.2    3.7       13.8    7.6
196   94.2    4.9        8.1    9.7
197  177.0    9.3        6.4   12.8
198  283.6   42.0       66.2   25.5
199  232.1    8.6        8.7   13.4

[200 rows x 4 columns]

In [3]:
import torch
from torch.optim import SGD
from sklearn.model_selection import train_test_split

torch.manual_seed(124)

features, targets = pre_a_df.iloc[:, :-1], pre_a_df.iloc[:, -1]

X_train, X_test, y_train, y_test = \
train_test_split(features, targets, test_size=0.2, random_state=124)

X_train1 = torch.FloatTensor(X_train.TV.values).view(-1, 1)
X_train2 = torch.FloatTensor(X_train.Radio.values).view(-1, 1)
X_train3 = torch.FloatTensor(X_train.Newspaper.values).view(-1, 1)
y_train = torch.FloatTensor(y_train.values).view(-1, 1)

X_test1 = torch.FloatTensor(X_test.TV.values).view(-1, 1)
X_test2 = torch.FloatTensor(X_test.Radio.values).view(-1, 1)
X_test3 = torch.FloatTensor(X_test.Newspaper.values).view(-1, 1)
y_test = torch.FloatTensor(y_test.values).view(-1, 1)

W1 = torch.zeros(1, requires_grad=True)
W2 = torch.zeros(1, requires_grad=True)
W3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = SGD([W1, W2, W3, b], lr=1e-5)

epochs = 1000

for epoch in range(1, epochs + 1):
    H = W1 * X_train1 + W2 * X_train2 + W3 * X_train3 + b
    loss = torch.mean((y_train - H) ** 2)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('{:4d}/{}: W1: {:.4f}, W2: {:.4f}, W3: {:.4f}, b: {:.4f}, loss: {:.4f}'\
              .format(epoch, epochs, W1.item(), W2.item(), W3.item(), b.item(), loss.item()))

 100/1000: W1: 0.0616, W2: 0.1041, W3: 0.0580, b: 0.0025, loss: 7.4172
 200/1000: W1: 0.0561, W2: 0.1499, W3: 0.0570, b: 0.0038, loss: 5.1731
 300/1000: W1: 0.0543, W2: 0.1762, W3: 0.0482, b: 0.0048, loss: 4.3777
 400/1000: W1: 0.0535, W2: 0.1929, W3: 0.0402, b: 0.0057, loss: 4.0271
 500/1000: W1: 0.0531, W2: 0.2039, W3: 0.0342, b: 0.0066, loss: 3.8664
 600/1000: W1: 0.0529, W2: 0.2113, W3: 0.0300, b: 0.0074, loss: 3.7921
 700/1000: W1: 0.0528, W2: 0.2162, W3: 0.0271, b: 0.0082, loss: 3.7576
 800/1000: W1: 0.0527, W2: 0.2196, W3: 0.0251, b: 0.0090, loss: 3.7414
 900/1000: W1: 0.0526, W2: 0.2219, W3: 0.0238, b: 0.0098, loss: 3.7337
1000/1000: W1: 0.0525, W2: 0.2234, W3: 0.0228, b: 0.0106, loss: 3.7298


In [18]:
pre_a_df.corr()['Sales'].sort_values(ascending=False)[1:]

TV           0.782224
Radio        0.576223
Newspaper    0.228299
Name: Sales, dtype: float64

In [17]:
pre_a_df

TV  Radio  Newspaper  Sales
0    230.1   37.8       69.2   22.1
1     44.5   39.3       45.1   10.4
2     17.2   45.9       69.3    9.3
3    151.5   41.3       58.5   18.5
4    180.8   10.8       58.4   12.9
..     ...    ...        ...    ...
195   38.2    3.7       13.8    7.6
196   94.2    4.9        8.1    9.7
197  177.0    9.3        6.4   12.8
198  283.6   42.0       66.2   25.5
199  232.1    8.6        8.7   13.4

[200 rows x 4 columns]

In [4]:
H = 0.0525 * X_test1 + 0.2234 * X_test2 + 0.0228 * X_test3 + 0.0106
loss = torch.mean((y_test - H) ** 2)
print(loss.item())

5.411946773529053


In [5]:
import torch
from torch.optim import SGD
from sklearn.model_selection import train_test_split

torch.manual_seed(124)

features, targets = pre_a_df.iloc[:, :-1], pre_a_df.iloc[:, -1]

X_train, X_test, y_train, y_test = \
train_test_split(features, targets, test_size=0.2, random_state=124)

X_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(y_train.values).view(-1, 1)

X_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(y_test.values).view(-1, 1)

W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = SGD([W, b], lr=1e-5)

epochs = 1000

for epoch in range(1, epochs + 1):
    H = X_train.matmul(W) + b
    loss = torch.mean((y_train - H) ** 2)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('{:4d}/{}: W1: {:.4f}, W2: {:.4f}, W3: {:.4f}, b: {:.4f}, loss: {:.4f}'\
              .format(epoch, epochs, W[0].item(), W[1].item(), W[2].item(), b.item(), loss.item()))

 100/1000: W1: 0.0616, W2: 0.1041, W3: 0.0580, b: 0.0025, loss: 7.4172
 200/1000: W1: 0.0561, W2: 0.1499, W3: 0.0570, b: 0.0038, loss: 5.1731
 300/1000: W1: 0.0543, W2: 0.1762, W3: 0.0482, b: 0.0048, loss: 4.3777
 400/1000: W1: 0.0535, W2: 0.1929, W3: 0.0402, b: 0.0057, loss: 4.0271
 500/1000: W1: 0.0531, W2: 0.2039, W3: 0.0342, b: 0.0066, loss: 3.8664
 600/1000: W1: 0.0529, W2: 0.2113, W3: 0.0300, b: 0.0074, loss: 3.7921
 700/1000: W1: 0.0528, W2: 0.2162, W3: 0.0271, b: 0.0082, loss: 3.7576
 800/1000: W1: 0.0527, W2: 0.2196, W3: 0.0251, b: 0.0090, loss: 3.7414
 900/1000: W1: 0.0526, W2: 0.2219, W3: 0.0238, b: 0.0098, loss: 3.7337
1000/1000: W1: 0.0525, W2: 0.2234, W3: 0.0228, b: 0.0106, loss: 3.7298


In [6]:
H = 0.0525 * X_test1 + 0.2234 * X_test2 + 0.0228 * X_test3 + 0.0106
loss = torch.mean((y_test - H) ** 2)
print(loss.item())

5.411946773529053


In [7]:
import torch
from torch.nn import Linear
from torch.nn.functional import mse_loss
from torch.optim import SGD
from sklearn.model_selection import train_test_split

torch.manual_seed(124)

features, targets = pre_a_df.iloc[:, :-1], pre_a_df.iloc[:, -1]

X_train, X_test, y_train, y_test = \
train_test_split(features, targets, test_size=0.2, random_state=124)

X_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(y_train.values).view(-1, 1)

X_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(y_test.values).view(-1, 1)

l_r = Linear(3, 1)

optimizer = SGD(l_r.parameters(), lr=1e-5)

epochs = 1000

for epoch in range(1, epochs + 1):
    H = l_r(X_train)
    loss = mse_loss(y_train, H)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('{:4d}/{}: '\
              .format(epoch, epochs, W[0].item(), W[1].item(), W[2].item(), b.item(), loss.item()), end='')
        for i, w in enumerate(list(l_r.parameters())[0][0]):
            print('W{}: {:.4f}, '\
                  .format(i + 1, w.item()), end='')
        print('b: {:.4f}, loss: {:.4f}'.format(list(l_r.parameters())[1].item(), loss.item()))

 100/1000: W1: 0.0375, W2: 0.1320, W3: 0.1850, b: -0.0293, loss: 15.3034
 200/1000: W1: 0.0503, W2: 0.1429, W3: 0.0983, b: -0.0288, loss: 6.6044
 300/1000: W1: 0.0531, W2: 0.1649, W3: 0.0647, b: -0.0280, loss: 4.9029
 400/1000: W1: 0.0535, W2: 0.1836, W3: 0.0485, b: -0.0271, loss: 4.2727
 500/1000: W1: 0.0533, W2: 0.1973, W3: 0.0391, b: -0.0262, loss: 3.9925
 600/1000: W1: 0.0531, W2: 0.2068, W3: 0.0332, b: -0.0253, loss: 3.8638
 700/1000: W1: 0.0529, W2: 0.2133, W3: 0.0292, b: -0.0245, loss: 3.8043
 800/1000: W1: 0.0528, W2: 0.2177, W3: 0.0266, b: -0.0237, loss: 3.7766
 900/1000: W1: 0.0527, W2: 0.2207, W3: 0.0248, b: -0.0229, loss: 3.7635
1000/1000: W1: 0.0527, W2: 0.2228, W3: 0.0236, b: -0.0221, loss: 3.7571


In [8]:
H = 0.0527 * X_test1 + 0.2228 * X_test2 + 0.0236 * X_test3 + -0.0221
loss = torch.mean((y_test - H) ** 2)
print(loss.item())

5.448765277862549


In [9]:
from torch.nn import Module, Linear

class LinearRegressionModel(Module):
    def __init__(self):
        super().__init__()
        self.linear = Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [10]:
import torch
from torch.nn.functional import mse_loss
from torch.optim import SGD
from sklearn.model_selection import train_test_split

torch.manual_seed(124)

features, targets = pre_a_df.iloc[:, :-1], pre_a_df.iloc[:, -1]

X_train, X_test, y_train, y_test = \
train_test_split(features, targets, test_size=0.2, random_state=124)

X_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(y_train.values).view(-1, 1)

X_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(y_test.values).view(-1, 1)

l_r = LinearRegressionModel()

optimizer = SGD(l_r.parameters(), lr=1e-5)

epochs = 1000

for epoch in range(1, epochs + 1):
    H = l_r(X_train)
    loss = mse_loss(y_train, H)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('{:4d}/{}: '\
              .format(epoch, epochs, W[0].item(), W[1].item(), W[2].item(), b.item(), loss.item()), end='')
        for i, w in enumerate(list(l_r.parameters())[0][0]):
            print('W{}: {:.4f}, '\
                  .format(i + 1, w.item()), end='')
        print('b: {:.4f}, loss: {:.4f}'.format(list(l_r.parameters())[1].item(), loss.item()))

 100/1000: W1: 0.0375, W2: 0.1320, W3: 0.1850, b: -0.0293, loss: 15.3034
 200/1000: W1: 0.0503, W2: 0.1429, W3: 0.0983, b: -0.0288, loss: 6.6044
 300/1000: W1: 0.0531, W2: 0.1649, W3: 0.0647, b: -0.0280, loss: 4.9029
 400/1000: W1: 0.0535, W2: 0.1836, W3: 0.0485, b: -0.0271, loss: 4.2727
 500/1000: W1: 0.0533, W2: 0.1973, W3: 0.0391, b: -0.0262, loss: 3.9925
 600/1000: W1: 0.0531, W2: 0.2068, W3: 0.0332, b: -0.0253, loss: 3.8638
 700/1000: W1: 0.0529, W2: 0.2133, W3: 0.0292, b: -0.0245, loss: 3.8043
 800/1000: W1: 0.0528, W2: 0.2177, W3: 0.0266, b: -0.0237, loss: 3.7766
 900/1000: W1: 0.0527, W2: 0.2207, W3: 0.0248, b: -0.0229, loss: 3.7635
1000/1000: W1: 0.0527, W2: 0.2228, W3: 0.0236, b: -0.0221, loss: 3.7571


In [11]:
H = 0.0527 * X_test1 + 0.2228 * X_test2 + 0.0236 * X_test3 + -0.0221
loss = torch.mean((y_test - H) ** 2)
print(loss.item())

5.448765277862549


In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

features, targets = pre_a_df.iloc[:, :-1], pre_a_df.iloc[:, -1]

X_train, X_test, y_train, y_test = \
train_test_split(features, targets, test_size=0.2, random_state=124)

l_r = LinearRegression()
l_r.fit(X_train, y_train)
print('W: {:.4f}, b: {:.4f}'.format(l_r.coef_[0], l_r.intercept_))

W: 0.0448, b: 2.8103


In [13]:
from sklearn.metrics import mean_squared_error

prediction = l_r.predict(X_test)
print('MSE loss: {:.4f}, RMSE loss: {:.4f}'\
      .format(mean_squared_error(y_test, prediction), 
      np.sqrt(mean_squared_error(y_test, prediction))))

MSE loss: 3.5016, RMSE loss: 1.8713
